In [ ]:
%matplotlib notebook

import sys
from typing import List, Tuple
from configparser import ConfigParser
import random
import time

config = ConfigParser()
config.read("config.ini")
CARLA_PATH = config.get("main", "CARLA_PATH")

# Enable import of 'carla'
sys.path.append(CARLA_PATH + "PythonAPI/carla/dist/carla-0.9.9-py3.6-linux-x86_64.egg")
# Enable import of 'agents' and it's submodules
sys.path.insert(0, CARLA_PATH + "PythonAPI/carla/")
# Enable import of utilities from GlobalPathPlanner
sys.path.insert(0, "../../../global_route_planner/")

import numpy as np
import carla
from agents.navigation.local_planner import LocalPlanner, _compute_connection
from shapely.geometry import LineString, Point
from matplotlib import pyplot as plt
from IPython import display


from global_planner import get_client, spawn_vehicle, draw_waypoints, get_global_planner
from cartesian_to_frenet import get_frenet_from_cartesian, get_path_linestring

### Get carla interface utils

In [ ]:
# Get client to interact with CARLA server
client = get_client()

# Get current CARLA world
world = client.get_world()

## Options to control aspects of this example

In [ ]:
opt_dict = {"target_speed": 30, "num_waypoints_in_lane": 10000}

## Get two points on the map, these act as the start and end points of a global route.

In [ ]:
start_point = random.choice(world.get_map().get_spawn_points())
end_point = random.choice(world.get_map().get_spawn_points())

## Get a path between the two points


In [ ]:
planner_resolution = opt_dict['target_speed'] * 0.1 / 3.6  # 0.1 seconds horizon
grp = get_global_planner(world=world, planner_resolution=planner_resolution)

route = grp.trace_route(start_point.location, end_point.location)
route_waypoints = [route[i][0] for i in range(len(route))]

# Visualize the path in CARLA world
draw_waypoints(world, route_waypoints, 200)

## Spawn a vehicle at the start point, and attach a local planner to it

In [ ]:
vehicle.destroy()
vehicle_query.destroy()
vehicle = spawn_vehicle(world=world, spawn_point=start_point)
vehicle_query = spawn_vehicle(world=world, spawn_point=end_point)


local_planner = LocalPlanner(vehicle, opt_dict=opt_dict)

## Set the global path as the desired trajectory of the local planner

In [ ]:
local_planner.set_global_plan(route)

# Visualize the path in CARLA world
draw_waypoints(world, route_waypoints, 200)

## Control 

In [ ]:
fig,ax = plt.subplots(1,1)


def get_vehicle_state(vehicle):
    vehicle_location = vehicle.get_location()
    x = vehicle_location.x
    y = vehicle_location.y
    heading = vehicle.get_transform().rotation.yaw
    return x, y, heading


while not local_planner.done():

    
    len_waypoint_buffer = len(local_planner._waypoint_buffer)
    lane_waypoints_with_roadoptions = list(local_planner._waypoint_buffer) + list(local_planner._waypoints_queue)[:opt_dict['num_waypoints_in_lane']-len_waypoint_buffer]
    lane_waypoints = [item[0] for item in lane_waypoints_with_roadoptions]
    
    xs = [waypoint.transform.location.x for waypoint in lane_waypoints]
    ys = [waypoint.transform.location.y for waypoint in lane_waypoints]

    # Get Vehicle State Information
    x,y,heading = get_vehicle_state(vehicle)

    x_q,y_q,heading_q = get_vehicle_state(vehicle_query)


    linestring = get_path_linestring(lane_waypoints)
    
    s_q, d_q, frenet_heading_q, _ = get_frenet_from_cartesian(linestring, Point(x_q,y_q), heading_q)
    s, d, frenet_heading, _ = get_frenet_from_cartesian(linestring, Point(x,y), heading)

    print("Frenet - Query Vehicle:", "s:", s_q, "d:", d_q, "heading:", frenet_heading_q)
    print("Frenet - Ego Vehicle:", "s:",s, "d:", d, "heading:",frenet_heading) 
    
    control = local_planner.run_step(debug=True)
    vehicle.apply_control(control)

    
    display.clear_output(wait=True)
    

